In [111]:
import dash
import pandas as pd
import plotly.graph_objects as go
from dash import dcc, html, dash_table 
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import json
from dash import Input, Output, ctx
import numpy as np
import requests
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
from scipy.stats import pearsonr

In [258]:
df=pd.read_excel('base_file.xlsx')
with open("municipalities.geojson", "r", encoding="utf-8") as f:
    denmark_geojson = json.load(f)

In [259]:
#=========File structure and basic info=========

print("DATAFRAME INFO:")
print(f"Shape: {df.shape} (rows, columns)")
print(f"\nColumn Names:\n{df.columns.tolist()}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nFirst few rows:\n{df.head()}")
print(f"\nUnique values in key columns:")
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"  {col}: {df[col].unique()[:5]}")


DATAFRAME INFO:
Shape: (90, 18) (rows, columns)

Column Names:
['Municipality', 'family_1', 'family_2', 'family_3', 'family_3+', 'income_23', 'population_25', 'normal_charger', 'fast_charger', 'new_electric24', 'new_petrol24', 'new_disel24', 'stock_petrol25', 'stock_disel25', 'stock_electric25', 'house_multidwelling', 'house_detached', 'higher_education']

Data Types:
Municipality           object
family_1                int64
family_2                int64
family_3                int64
family_3+               int64
income_23               int64
population_25           int64
normal_charger          int64
fast_charger            int64
new_electric24          int64
new_petrol24            int64
new_disel24             int64
stock_petrol25          int64
stock_disel25           int64
stock_electric25        int64
house_multidwelling     int64
house_detached          int64
higher_education        int64
dtype: object

First few rows:
  Municipality  family_1  family_2  family_3  family_3+  i

In [260]:
# ========= Metrics on Municipality level==========

# Total stock and new cars per municipality
df["stock_total25"] = df["stock_petrol25"] + df["stock_disel25"] + df["stock_electric25"]
df["new_total24"]   = df["new_petrol24"]   + df["new_disel24"]   + df["new_electric24"]

# Shares of EVs (stock and new registrations)
df["ev_share_stock"] = df["stock_electric25"] / df["stock_total25"]
df["ev_share_new24"] = df["new_electric24"]   / df["new_total24"]

# EVs per 1,000 inhabitants
df["ev_per_1000"] = 1000 * df["stock_electric25"] / df["population_25"]

# Chargers per 1,000 inhabitants
df["chargers_total"] = df["normal_charger"] + df["fast_charger"]
df["chargers_per_1000"] = 1000 * df["chargers_total"] / df["population_25"]

# Share of fast chargers
df["fast_share_chargers"] = df["fast_charger"] / df["chargers_total"]

# Share of electric cars to fast chargers
df["ev_to_fast_chargers"] = round(( df["stock_electric25"] / df["fast_charger"]),2)

#EV's per normal charger
df["ev_to_normal_chargers"] = round(( df["stock_electric25"] / df["normal_charger"]),2)

#EV's per total chargers
df["evs_to_chargers"] = round(( df["stock_electric25"] / df["chargers_total"]),2)

# Municipality-level KPIs for hover tooltips

# 1) EV share of total stock (%)
df["ev_share_stock_ml"] = (df["stock_electric25"] / df["stock_total25"] * 100).round(1)

# 2) EV share of new cars 2024 (%)
df["ev_share_new24_ml"] = (df["new_electric24"] / df["new_total24"] * 100).round(1)

# 3) EVs per charger
df["evs_per_charger_ml"] = (df["stock_electric25"] / df["chargers_total"]).round(1)

# 4) EVs per fast charger
df["evs_per_fast_charger_ml"] = (df["stock_electric25"] / df["fast_charger"]).round(1)

# 5) EVs per 1,000 people
df["evs_per_1000_ml"] = (1000 * df["stock_electric25"] / df["population_25"]).round(1)

#Socioeconomic variables (%)
df["house_total"] = df["house_detached"] + df["house_multidwelling"]
df["house_detached_pct"] = (df["house_detached"] / df["house_total"] * 100)
df["house_multidwelling_pct"] = (df["house_multidwelling"] / df["house_total"] * 100)

df["households_total"] = df["family_1"] + df["family_2"] + df["family_3"] + df["family_3+"]
df["family_1_pct"] = df["family_1"] / df["households_total"] * 100
df["family_3_pct"] = df["family_3"] / df["households_total"] * 100
df["family_2_pct"] = df["family_2"] / df["households_total"] * 100
df["family_3plus_pct"] = df["family_3+"] / df["households_total"] * 100

df["higher_education_pct"] = (df["higher_education"] / df["population_25"] * 100)


In [264]:
# ============KPIs for cards==========

# 1) Electric cars share of total stock
electric_ratio = (df["stock_electric25"].sum() / df["stock_total25"].sum() * 100).round(1)

# 2) Growth ratio 2024: new EVs vs all new cars
electric_growth_ratio = (df["new_electric24"].sum() / df["new_total24"].sum() * 100).round(1)

# 3) EVs per charger (all chargers)
ev_to_chargers = (df["stock_electric25"].sum() / df["chargers_total"].sum()).round(1)

# 4) EVs per fast charger
ev_to_fast_chargers = (df["stock_electric25"].sum() / df["fast_charger"].sum()).round(1)

# 5) EVs per 1,000 people (using the new column)
evs_per_1000 = (1000 * df["stock_electric25"].sum() / df["population_25"].sum()).round(1)


In [266]:
# --- Battery KPI component ---

def battery_kpi(title, slug, display_value, unit="", fill_pct=None, show_fill=True, fill_color="#33C57A"):
    """
    Battery KPI component.
    - slug: unique string like "ev_share_stock"
    - display_value: number shown inside battery
    - fill_pct: 0..100 (controls fill width). If None, uses display_value as pct.
    - show_fill: True for % KPIs, False for ratio KPIs (no fill)
    """
    # fill level
    if fill_pct is None:
        try:
            fill = float(display_value)
        except:
            fill = 0
    else:
        fill = float(fill_pct)

    fill = max(0, min(100, fill))

    battery_wrap = {
        "position": "relative",
        "width": "230px",
        "height": "70px",
        "border": "3px solid #1f1f1f",
        "borderRadius": "14px",
        "background": "rgba(255,255,255,0.40)",
        "overflow": "hidden",
        "marginTop": "10px"
    }

    fill_style = {
        "height": "100%",
        "width": f"{fill}%",
        "background": f"linear-gradient(90deg, {fill_color}, #1f1f1f22)",
        "borderRadius": "12px 0 0 12px",
        "transition": "width 0.3s ease"
    }

    cap_style = {
        "position": "absolute",
        "right": "-14px",
        "top": "22px",
        "width": "14px",
        "height": "26px",
        "border": "3px solid #1f1f1f",
        "borderLeft": "0px",
        "borderRadius": "0 8px 8px 0",
        "background": "rgba(255,255,255,0.12)"
    }

    text_style = {
        "position": "absolute",
        "top": "50%",
        "left": "50%",
        "transform": "translate(-50%, -50%)",
        "fontWeight": "700",
        "fontSize": "27px",
        "color": "white"
    }

    return html.Div([
        html.H4(["⚡ ", title], style=label_style),

        html.Div([
            html.Div(id=f"kpi-{slug}-fill", style=fill_style) if show_fill else None,
            html.Div(style=cap_style),
            html.Div(id=f"kpi-{slug}-text", children=f"{display_value}{unit}", style=text_style),
        ], style=battery_wrap),

    ], style=tile_style)


In [268]:
#=========Map-EV distribution==========

# Dropdown metrics
map_metrics = {
    "stock_electric25": "EV total stock",
    "new_electric24": "New EV's in 2024",
    "evs_to_chargers": "EV's to chargers"
}
#setting green scale for legend
green_scale = [
    [0.0, "#FFFFFF"],   
    [0.4, "#A9EFC5"],   
    [0.7, "#33C57A"],   
    [1.0, "#008C45"]
]


def make_map(df, metric, denmark_geojson, highlight=None):
    fig = px.choropleth_mapbox(
        df,
        geojson=denmark_geojson,
        locations="Municipality",
        featureidkey="properties.label_dk",
        color=metric,
        color_continuous_scale=green_scale,
        mapbox_style="carto-positron",
        zoom=6,
        center={"lat": 56.0, "lon": 10.5},
        opacity=0.8,
        custom_data=[
            "Municipality",
            "ev_share_stock_ml",
            "ev_share_new24_ml",
            "evs_per_charger_ml",
            "evs_per_fast_charger_ml",
            "evs_per_1000_ml",
        ]
    )

    # Apply hovertemplate ONLY to the base choropleth
    fig.update_traces(
        hovertemplate=(
            "<b>%{customdata[0]}</b><br><br>"
            "EV share of stock: %{customdata[1]:.1f}%<br>"
            "EV share of new cars 2024: %{customdata[2]:.1f}%<br>"
            "EVs per charger: %{customdata[3]:.1f}<br>"
            "EVs per fast charger: %{customdata[4]:.1f}<br>"
            "EVs per 1,000 people: %{customdata[5]:.1f}"
            "<extra></extra>"
        ),
        selector=dict(type="choroplethmapbox")
    )

    # Highlight selected municipality (outline only, no hover)
    if highlight:
        fig.add_trace(go.Choroplethmapbox(
            geojson=denmark_geojson,
            locations=[highlight],
            z=[1],
            colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
            marker_line_width=2,
            marker_line_color="#FFD700",
            showscale=False,
            featureidkey="properties.label_dk",
            hoverinfo="skip",
            hovertemplate=None
        ))

    #  Enable clicking
    fig.update_layout(clickmode="event")

    return fig

In [270]:
# ==========Bar chart: EV's to fast chargers TOP5==========


def make_bar(df, metric, mode):
    """ Return top 10 bar displaying top 10 or bottom 10 municipalities for the chosen metric"""

    title_metric = map_metrics.get(metric, metric)

    if mode == "bottom":
        subset = df.sort_values(metric, ascending=True).head(10)
        title = f"Bottom 10 municipalities by {title_metric.lower()}"
    else:
        subset = df.sort_values(metric, ascending=False).head(10)
        title = f"Top 10 municipalities by {title_metric.lower()}"

    fig = px.bar(
        subset,
        x="Municipality",
        y=metric,
        text=metric,
        title=title,
    )

    fig.update_traces(
        marker_color="#008c4a",
        texttemplate="%{text:,.0f}",
        textposition="outside"
    )

    fig.update_layout(
        yaxis_title=title_metric,
        xaxis_title="Municipality",
        uniformtext_minsize=12,
        uniformtext_mode="hide",
        margin={"t": 60, "l": 40, "r": 20, "b": 40},
        plot_bgcolor="rgb(249, 249, 249)",
        paper_bgcolor="rgb(249, 249, 249)"
    )
    return fig


In [272]:
#==========EV's share table==========

def make_top5_ev_share_table(df, selected_municipality=None):
    """Return top 5 table OR selected municipality stats in the same table style."""

    dff = df.copy()
    dff["ev_share_stock_pct"] = (dff["ev_share_stock"] * 100).round(1)
    dff["ev_share_new24_pct"] = (dff["ev_share_new24"] * 100).round(1)

    # --- CASE 1: No municipality selected → return normal Top 5 ---
    if selected_municipality is None:
        top5 = (
            dff.sort_values("ev_share_stock", ascending=False)
               .loc[:, ["Municipality", "ev_share_stock_pct", "ev_share_new24_pct"]]
               .head(5)
        )
        data = top5.to_dict("records")

    # --- CASE 2: Municipality selected → return a 1-row table with same structure ---
    else:
        row = dff[dff["Municipality"] == selected_municipality].iloc[0]
        data = [{
            "Municipality": row["Municipality"],
            "ev_share_stock_pct": row["ev_share_stock_pct"],
            "ev_share_new24_pct": row["ev_share_new24_pct"],
        }]

    # --- Return the SAME table structure ---
    return dash_table.DataTable(
        id="ev-share-table",
        columns=[
            {"name": "Municipality", "id": "Municipality"},
            {"name": "EV Share (Stock %)", "id": "ev_share_stock_pct"},
            {"name": "EV Share (New %)", "id": "ev_share_new24_pct"},
        ],
        data=data,
        style_cell={
            "textAlign": "left",
            "padding": "6px",
            "fontFamily": "Arial",
            "fontSize": "14px",
        },
        style_header={
            "backgroundColor": "#00a651",
            "color": "white",
            "fontWeight": "bold"
        },
        page_size=5,
        style_table={"marginTop": "10px", "width": "100%"}
    )



In [274]:
#==========Variable Scatter PLot==========

# Labels for the dropdown + axis titles
scatter_label_map = {
    "income_23": "Average income (2023, DKK)",
    "higher_education_pct": "Higher education (%)",
    "house_detached_pct": "Detached housing (%)",
    "house_multidwelling_pct": "Multidwelling housing (%)",
    "family_1_pct": "One-person households (%)",
    "family_2_pct": "Two-person households (%)",
    "family_3_pct": "Three-person households (%)",
    "family_3plus_pct": "Three+ person households (%)",
}

def make_scatter(df, x_col, outlier_mode="include"):
    y_col = "ev_per_1000"

    # Base data (all points for scatter)
    dff = df[[x_col, y_col, "Municipality"]].replace(
        [np.inf, -np.inf], np.nan
    ).dropna()

    fig = px.scatter(
        dff,
        x=x_col,
        y=y_col,
        hover_name="Municipality",
        labels={
            x_col: scatter_label_map.get(x_col, x_col),
            y_col: "EVs per 1,000 inhabitants"
        },
        color_discrete_sequence=["#33C57A"],
        template="simple_white"
    )

    # Style points
    fig.update_traces(
        marker=dict(size=9, opacity=0.75, line=dict(width=0.5, color="white")),
        selector=dict(mode="markers")
    )

    # -------- Trendline data (may exclude outliers) --------
    trend_df = dff.copy()

    if outlier_mode == "exclude":
        cutoff = trend_df[x_col].quantile(0.90)
        trend_df = trend_df[trend_df[x_col] <= cutoff]

    # Fit linear regression
    X = trend_df[[x_col]].values
    y = trend_df[y_col].values

    model = LinearRegression()
    model.fit(X, y)

    x_range = np.linspace(trend_df[x_col].min(), trend_df[x_col].max(), 200)
    y_pred = model.predict(x_range.reshape(-1, 1))

    # Add trendline
    fig.add_trace(
        px.line(
            x=x_range,
            y=y_pred
        ).data[0]
    )

    fig.update_traces(
        line=dict(color="#2C7A7B", width=2),
        selector=dict(mode="lines")
    )

    # -------- Correlation metrics (same data as trendline) --------
    r, _ = pearsonr(trend_df[x_col], trend_df[y_col])
    r2 = r ** 2

    fig.add_annotation(
        x=0.02,
        y=0.98,
        xref="paper",
        yref="paper",
        showarrow=False,
        align="left",
        text=f"Pearson r = {r:.2f}<br>R² = {r2:.2f}",
        font=dict(size=13),
        bgcolor="rgba(255,255,255,0.75)",
        borderpad=6
    )

    # Title
    suffix = " (excluding top 10% outliers)" if outlier_mode == "exclude" else ""
    fig.update_layout(
        height=450,
        title=dict(
            text=f"{scatter_label_map.get(x_col, x_col)} vs EV adoption{suffix}",
            x=0.5,
            xanchor="center"
        )
    )

    return fig


In [276]:
#==========Quadrant Chart==========
def make_quadrant_chart(df, x_col, y_col, x_label, y_label, title, outlier_quantile=None):
    df = df.copy()
    """Return quadrant chart to evaluate the municipality infrastructure"""


    if outlier_quantile is not None:
        cutoff = df[x_col].quantile(outlier_quantile)
        df = df[df[x_col] <= cutoff]

    # Medians 
    x_mid = df[x_col].median()
    y_mid = df[y_col].median()
    
    # Assign quadrants
    def quadrant_label(row):
        if row[x_col] > x_mid and row[y_col] > y_mid:
            return "Strained momentum"
        elif row[x_col] <= x_mid and row[y_col] > y_mid:
            return "Efficient adopters"
        elif row[x_col] > x_mid and row[y_col] <= y_mid:
            return "At risk"
        else:
            return "Prepared but slow"

    df["quadrant"] = df.apply(quadrant_label, axis=1)

    # Colors for each quadrant
    color_map = {
        "Efficient adopters": "#1b9e77",   # green-ish
        "Strained momentum": "#d95f02",   # orange
        "Prepared but slow": "#7570b3", # blue/purple
        "At risk": "#e7298a",   # red/pink
    }

    # Base scatter plot
    fig = px.scatter(
        df,
        x=x_col,
        y=y_col,
        color="quadrant",
        color_discrete_map=color_map,
        hover_name="Municipality",
        labels={x_col: x_label, y_col: y_label},
        template="simple_white",
    )

    fig.update_traces(
        marker=dict(size=10, line=dict(width=1, color="white")),
        opacity=0.8
    )

    # Median lines
    fig.add_shape(
        type="line",
        x0=x_mid, x1=x_mid,
        y0=df[y_col].min(), y1=df[y_col].max(),
        line=dict(color="grey", dash="dash", width=1),
        layer="below"
    )
    fig.add_shape(
        type="line",
        x0=df[x_col].min(), x1=df[x_col].max(),
        y0=y_mid, y1=y_mid,
        line=dict(color="grey", dash="dash", width=1),
        layer="below"
    )

    # Quadrant labels
    x_min, x_max = df[x_col].min(), df[x_col].max()
    y_min, y_max = df[y_col].min(), df[y_col].max()

    centers = {
        "Strained momentum":   ((x_mid + x_max) / 2, (y_mid + y_max) / 2),
        "Efficient adopters":   ((x_min + x_mid) / 2, (y_mid + y_max) / 2),
        "At risk": ((x_mid + x_max) / 2, (y_min + y_mid) / 2),
        "Prepared but slow":   ((x_min + x_mid) / 2, (y_min + y_mid) / 2),
    }
    
    
    for label, (xc, yc) in centers.items():
  
      shift = 0
      if label == "At risk":
         shift = -70   
      elif label == "Prepared but slow":
         shift = -70   
     

      fig.add_annotation(
        x=xc,
        y=yc,
        text=label,
        showarrow=False,
        font=dict(color=color_map[label], size=14, family="Arial"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor=color_map[label],
        borderwidth=1,
        borderpad=4,
        yshift=shift
    )
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor="center",
            font=dict(size=20, family="Arial")
        ),showlegend=False,
        plot_bgcolor="rgb(249, 249, 249)",
        paper_bgcolor="rgb(249, 249, 249)"
    )

    return fig




In [284]:

app = dash.Dash(__name__)

container_style = {
    "backgroundColor": "#00a651",
    "color": "white",
    "borderRadius": "30px",
    "padding": "20px",
    "margin": "20px",
    "display": "flex",
    "justifyContent": "space-around",
    "alignItems": "center",
    "boxShadow": "2px 2px 8px rgba(0,0,0,0.2)"
}

tile_style = {
    "display": "flex",
    "flexDirection": "column", 
    "alignItems": "center",
    "margin": "0 15px"
}

label_style = {
    "fontSize": "20px",
    "margin": "0"
}

number_style = {
    "fontSize": "25px",
    "fontWeight": "bold",
    "margin": "5px 0 0 0"
}


app.layout = html.Div([
    html.H1("EV Infrastructure Dashboard", style={"textAlign": "center", "fontSize": "32px", "fontWeight": "bold", "fontFamily": "Segoe UI"}
),

    # ---- Row 1: KPI banner ---
   html.Div([
        battery_kpi("Electric Cars Share", "ev_share_stock", f"{electric_ratio:.1f}", unit="%", fill_pct=electric_ratio, show_fill=True),
        battery_kpi("EV Share of New Cars (2024)", "ev_share_new", f"{electric_growth_ratio:.1f}", unit="%", fill_pct=electric_growth_ratio, show_fill=True),
        battery_kpi("EVs per Charger", "ev_per_charger", f"{ev_to_chargers:.1f}", show_fill=False),
        battery_kpi("EVs per Fast Charger", "ev_per_fast", f"{ev_to_fast_chargers:.1f}", show_fill=False),
        battery_kpi("EVs per 1,000 People", "ev_per_1000", f"{evs_per_1000:.1f}", show_fill=False),
    ], style=container_style),

    # --- Row 2: Map + Bar chart ---
 html.Div([
    html.Div([
        html.Label("Choose map metric:", style={"fontWeight": "bold"}),

        dcc.Dropdown(
            id="map-metric",
            value=list(map_metrics.keys())[0],
            options=[{"label": label, "value": key} for key, label in map_metrics.items()],
            clearable=False,
            style={"width": "100%", "marginBottom": "20px"},
        ),

        dcc.Graph(
            id="municipality-map",
            style={"height": "650px", "width": "850px"}
        ),

    ], style={"flex": "2", "padding": "20px"}),

    html.Div([
        html.Label("Show municipalities:", style={"fontWeight": "bold"}),

        dcc.RadioItems(
            id="bar-mode",
            options=[
                {"label": "Top 10", "value": "top"},
                {"label": "Bottom 10", "value": "bottom"},
            ],
            value="top",
            inline=True,
            style={"marginBottom": "20px"},
        ),

        dcc.Graph(
            id="top10-bar",
            style={"height": "450px", "width": "550px"}
        ),

        html.Div(
            id="ev-share-table-container",
            style={"marginTop": "10px"}
        ),

        html.Div([
            html.Label("Select municipality:", style={"fontWeight": "bold"}),

            dcc.Dropdown(
                id="municipality-select",
                options=[{"label": m, "value": m} for m in df["Municipality"].unique()],
                placeholder="Choose a municipality...",
                clearable=True,
                style={"width": "100%", "marginBottom": "10px"}
            ),

            html.Button(
                "Reset selection",
                id="reset-selection",
                n_clicks=0,
                style={
                    "marginBottom": "20px",
                    "backgroundColor": "#d9534f",
                    "color": "white",
                    "border": "none",
                    "padding": "8px 12px",
                    "borderRadius": "5px",
                    "cursor": "pointer"
                }
            ),

            html.Div(id="selected-municipality-table")

        ], style={
            "backgroundColor": "#F9F9F9",
            "padding": "15px",
            "borderRadius": "8px",
            "marginTop": "10px"
        })

    ], style={"flex": "1", "padding": "20px"})

], style={
    "display": "flex",
    "backgroundColor": "#F9F9F9",
    "padding": "10px",
    "borderRadius": "8px"
}),

    # --- Row 3: Full-width scatterplot (RQ2) ---
    html.Div([
        html.Label("Choose socio-economic factor:", style={"fontWeight": "bold"}),

        dcc.Dropdown(
            id="scatter-x-select",
            value="income_23",
            options=[{"label": label, "value": key} for key, label in scatter_label_map.items()],
            clearable=False,
            style={"width": "40%", "marginBottom": "20px"},
        ),
        dcc.RadioItems(
        id="scatter-outlier-toggle",
        options=[
            {"label": "Include outliers", "value": "include"},
            {"label": "Exclude top 10%", "value": "exclude"},
        ],
        value="include",
        inline=True,
        style={"marginBottom": "15px"}
    ),
        dcc.Graph(id="ev-scatter", style={"width": "100%", "height": "450px"}),
    ], style={"padding": "20px", "marginTop": "20px"}),

    # --- Row 5: Full-width quadrant chart (RQ3) ---
    html.Div([
        dcc.Graph(id="quadrant-chart", style={"width": "100%", "height": "500px"})
    ], style={"backgroundColor": "#F9F9F9", "padding": "10px", "borderRadius": "8px"}),

    html.Label("Show outliers:", style={"fontWeight": "bold"}),

    dcc.RadioItems(
        id="outlier-toggle",
        options=[
            {"label": "Include all", "value": "include"},
            {"label": "Exclude top 10%", "value": "exclude"},
        ],
        value="exclude",
        inline=True,
        style={"marginBottom": "20px"},
    ),
])


# --- Callbacks---

def normalize(series, value):
    """Higher is better → normal scaling 0–100."""
    min_v = series.min()
    max_v = series.max()
    if max_v == min_v:
        return 0
    return (value - min_v) / (max_v - min_v) * 100


@app.callback(
    Output("kpi-ev_share_stock-text", "children"),
    Output("kpi-ev_share_stock-fill", "style"),
    Output("kpi-ev_share_new-text", "children"),
    Output("kpi-ev_share_new-fill", "style"),
    Output("kpi-ev_per_charger-text", "children"),
    Output("kpi-ev_per_fast-text", "children"),
    Output("kpi-ev_per_1000-text", "children"),
    Input("municipality-select", "value"),
    Input("reset-selection", "n_clicks"),
)
def update_kpis(selected, reset_clicks):

    if ctx.triggered_id == "reset-selection":
        selected = None

    # pick row
    if selected and selected in df["Municipality"].values:
        row = df[df["Municipality"] == selected].iloc[0]
    else:
        row = df.mean(numeric_only=True)  # national average

    # raw values
    ev_share = row["ev_share_stock_ml"]
    new_ev_share = row["ev_share_new24_ml"]
    ev_per_charger = row["evs_per_charger_ml"]
    ev_per_fast = row["evs_per_fast_charger_ml"]
    ev_per_1000 = row["evs_per_1000_ml"]

    # fill percentages (ONLY for % KPIs)
    fill_ev_share = normalize(df["ev_share_stock_ml"], ev_share)
    fill_new_ev = normalize(df["ev_share_new24_ml"], new_ev_share)

    def fill_style(pct):
        return {
            "height": "100%",
            "width": f"{pct:.1f}%",
            "background": "linear-gradient(90deg, #33C57A, #1f1f1f22)",
            "transition": "width 0.3s ease",
        }

    return (
        f"{ev_share:.1f}%", fill_style(fill_ev_share),
        f"{new_ev_share:.1f}%", fill_style(fill_new_ev),

        f"{ev_per_charger:.1f}",
        f"{ev_per_fast:.1f}",
        f"{ev_per_1000:.1f}",
    )

@app.callback(
    Output("municipality-map", "figure"),
    Output("ev-share-table-container", "children"),
    Output("municipality-select", "value"),
    Input("map-metric", "value"),
    Input("municipality-select", "value"),
    Input("reset-selection", "n_clicks"),
    Input("municipality-map", "clickData"),   # <-- NEW
)
def update_map_and_table(metric, selected, reset_clicks, clickData):

    triggered = ctx.triggered_id

    # Reset button clears selection
    if triggered == "reset-selection":
        selected = None

    # Clicking the map sets selection
    elif triggered == "municipality-map" and clickData and "points" in clickData and len(clickData["points"]) > 0:
        pt = clickData["points"][0]

        clicked = (
            pt.get("location")
            or pt.get("hovertext")
            or pt.get("customdata")
            or pt.get("text")
        )

        # if customdata is list/tuple, take first element
        if isinstance(clicked, (list, tuple)) and len(clicked) > 0:
            clicked = clicked[0]

        # Only accept if it matches your municipality names
        if clicked in df["Municipality"].values:
            selected = clicked

    # Build map + table using selected municipality
    fig_map = make_map(df, metric, denmark_geojson, highlight=selected)
    table = make_top5_ev_share_table(df, selected_municipality=selected)

    return fig_map, table, selected


@app.callback(
    Output("top10-bar", "figure"),
    Input("map-metric", "value"),
    Input("bar-mode", "value"),
)
def update_bar(metric, bar_mode):
    fig_bar = make_bar(df, metric, bar_mode)
    return fig_bar


@app.callback(
    Output("ev-scatter", "figure"),
    Input("scatter-x-select", "value"),
    Input("scatter-outlier-toggle", "value")
)
def update_scatter(x_col, outlier_mode):
    return make_scatter(df, x_col, outlier_mode)


@app.callback(
    Output("quadrant-chart", "figure"),
    Input("outlier-toggle", "value")
)

def update_quadrant_chart(toggle):
    if toggle == "exclude":
        return make_quadrant_chart(
            df,
            x_col="evs_to_chargers",
            y_col="ev_share_stock",
            x_label="EVs per charger",
            y_label="EV share of total stock",
            title="Infrastructure pressure in Danish municipalities",
            outlier_quantile=0.90
        )
    else:
        return make_quadrant_chart(
            df,
            x_col="evs_to_chargers",
            y_col="ev_share_stock",   # keep consistent (share vs share)
            x_label="EVs per charger",
            y_label="EV share of total stock",
            title="Infrastructure pressure in Danish municipalities"
        )



if __name__ == "__main__":
    app.run_server(debug=True, port=8052)

# Open your dashboard here: http://127.0.0.1:8052/


## 